<b><center> Processing a text file containing flight data </b></center>

<small> <center>The purpose of this script is to extract flight data from a text file, store it to Excel format and creating a KML file to visualize the itinerary

This method was developped based on the text file structure

<b> 1. Importing relevant libraries

In [ ]:
import pandas as pd

import simplekml # To create a KML file

<b> 2. Opening the text file

In [ ]:

thefile = open("C:/Users/ilung/Downloads/flight_data.txt", "r") # this line loads the text file

data = thefile.read() # This line is to read the text file that was loaded

<b> 3. Preparing the text file for its conversion to a dataframe </b>

<small> After carefully looking at the text file content, we chose the option to convert it to a list. Each line of the text file will become an element of the newly created list

In [ ]:
data_into_list = data.split("\n") # this line convert the text file into a list

data_into_list = [x.replace (" ", "") for x in data_into_list] # deleting empty spaces from every element of the list to facilitate further processing

<small> The text file contains 3 set of data: "altitude", "GPS coordinates" and "height above takeoff". Each associated with a timestamp. 

We will now proceed to determine the position where each of these dataset start. This will help us creating sublist of each dataset.

In [ ]:
index_alt = data_into_list.index('"altitude":{') # this line will help us to determine line where altitude related data begin

index_gps = data_into_list.index('"gps":{') # this line will help us to determine line where GPS related data begin

index_height = data_into_list.index('"height_above_takeoff":{') # this line will help us to determine line where "height above takeoff" related data begin

<small> Creating sublist for altitude, GPS and "height above takeoff" related data

In [ ]:
altitude = data_into_list[index_alt:index_gps] # Creating a sublist containing altitude related data

gps = data_into_list[index_gps:index_height] # Creating a sublist containing GPS related data

height = data_into_list[index_height:] # Creating a sublist containing "height above takeoff" related data

<b> 4. Creating dataframes and saving them into Excel </b>

<small> We now proceed to create individual dataframes for each dataset using the previously created sublists

In [ ]:
# For altitude

alt_data = altitude[2:339]

alt_timestamps = altitude[341:677]

alt_timestamps = [s.replace('"', '') for s in alt_timestamps]

alt_timestamps = [s.replace(',', '') for s in alt_timestamps]

df_altitude = pd.DataFrame(zip(alt_timestamps, alt_data), columns = ["Timestamps", "Altitude"])

In [ ]:
# For GPS

gps_coord = gps[3:1351]

gps_coord = [s.replace(",", "") for s in gps_coord]

while "]" in gps_coord:
    gps_coord.remove("]")

while "[" in gps_coord:
    gps_coord.remove("[")

gps_lat  = gps_coord[::2]

gps_lon = gps_coord[1::2]


gps_timestamps = gps[1351:]

del gps_timestamps [0]

gps_timestamps = [s.replace('"', '') for s in gps_timestamps]

gps_timestamps = [s.replace(',', '') for s in gps_timestamps]

df_gps = pd.DataFrame(zip(gps_lat, gps_lon, gps_timestamps), columns = ["Latitude", "Longitude", "Timestamps"])

In [ ]:
# For height_above_takeoff

height_data = height[2:307]

height_data = [s.replace(',', '') for s in height_data]


height_timestamps = height[309:615]

height_timestamps = [s.replace('"', '') for s in height_timestamps]

height_timestamps = [s.replace(',', '') for s in height_timestamps]


df_height = pd.DataFrame(zip(height_timestamps, height_data), columns = ["Timestamps", "Height"])

<small> Here we will save the created dataframes into an Excel file, with each dataframe being in a separate sheet

In [ ]:
# Saving the dataframes into different sheet of the same Excel file

writer = pd.ExcelWriter("C:/Users/ilung/Downloads/flight_data.xlsx")

df_altitude.to_excel(writer, sheet_name='Sheet1', index= False)

df_gps.to_excel(writer, sheet_name='Sheet2', index= False)

df_height.to_excel(writer, sheet_name='Sheet3', index= False)

writer.save()

<b> 5. Creating the KML file from the Excel file

In [ ]:
# Read the Excel file into a Pandas DataFrame

df = pd.read_excel("C:/Users/ilung/Downloads/flight_data.xlsx", sheet_name = "Sheet2")

In [ ]:
# Generating a list of coordinates to produce the line

i = 0

coords = []

while i < 337:

    lat = df.iloc[i, 0]

    long = df.iloc[i,1]

    point = (lat, long)

    coords.append(point)

    i = i+1

In [ ]:
# Create a KML object
kml = simplekml.Kml()

# Creating a line from the coordinate
lin = kml.newlinestring(name= "Pathway", description="Flight itinerary",
                        coords = coords)

kml.save("C:/Users/ilung/Downloads/flight itinerary.kml")